# // TO DO LIST//

1. convert state names in stadium dataframe to using abbreviations using some replace or something function (lambdas)

2. Get nearest large city from weather API if data doesn't exist (i.e. East Rutherford, NJ)

use website: https://api.meteostat.net/#history with api key=ipeqdKHL

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))

In [2]:
import pickle
from bs4 import BeautifulSoup
import csv
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import requests
import lxml
import json
import os
api_key = 'ipeqdKHL'

In [3]:
dirName = 'HistoricalWeatherData'
if not os.path.exists(dirName):
    os.mkdir(dirName)
    print('Directory ', dirName, ' Created')
else:
    print('Directory ', dirName, ' already exists')

Directory  HistoricalWeatherData  already exists


In [12]:
def get_stadium_cities_wiki():
    url = 'https://en.wikipedia.org/wiki/List_of_current_National_Football_League_stadiums'
    table = pd.read_html(url)
    df = table[1]
    
    keep_cols = ['Name', 'Capacity', 'Location', 'Surface', 'Roof type', 'Team(s)', 'Opened']
    
    df = df[keep_cols]
    
    
    
    df['Capacity'] = df['Capacity'].astype(int)
    df['Surface'] = df['Surface'].replace(to_replace=r'\[\d{1,2}\]', value='', regex=True)
    df['Opened'] = df['Opened'].replace(to_replace=r'\[.*?\]', value='', regex=True)
    
    # need to fix this regex to properly divide the column into 2
#     df['Team(s)'] = df['Team(s)'].str.split(r'(.*[A-Za-z]+)([A-Z].*)', expand=True)
        
    # save dataframe to pickle file
    pd.to_pickle(df, './stadium_dataframe_from_wiki.pkl')

In [25]:
def get_stadium_cities_nonwiki():
    url = 'https://geojango.com/pages/list-of-nfl-teams'
    table = pd.read_html(url)
    
    df = table[0]
    df[['City', 'State']] = df['Arena Location'].str.extract('^(?P<City>.+?),\s(?P<State>.+)', expand=True)
    
    # save dataframe to pickle file
    pd.to_pickle(df, './stadium_dataframe_not_from_wiki.pkl')
    
    
def get_state_abbrevs():
    url2 = 'https://www.softschools.com/social_studies/state_abbreviations/'
    table2 = pd.read_html(url2)
    df2 = table2[2]
    df2['State'] = df2['State'].str.title()
    df2.set_index('State', inplace=True)
    pd.to_pickle(df2, './state_abbreviations.pkl')

In [26]:
get_stadium_cities_nonwiki()
get_state_abbrevs()

In [27]:
stadiums_df = pd.read_pickle('./stadium_dataframe_not_from_wiki.pkl')
stadiums_df

,Team Name,Arena Name,Arena Location,Seating Capacity,Opening Year,City,State
0,Arizona Cardinals,State Farm Stadium,"Glendale, Arizona",63400,2006,Glendale,Arizona
1,Atlanta Falcons,Mercedes-Benz Stadium,"Atlanta, Georgia",71000,2017,Atlanta,Georgia
2,Baltimore Ravens,M&T Bank Stadium,"Baltimore, Maryland",71008,1998,Baltimore,Maryland
3,Buffalo Bills,New Era Field,"Orchard Park, New York",71608,1973,Orchard Park,New York
4,Carolina Panthers,Bank of America Stadium,"Charlotte, North Carolina",75523,1996,Charlotte,North Carolina
5,Chicago Bears,Soldier Field,"Chicago, Illinois",61500,1924,Chicago,Illinois
6,Cincinnati Bengals,Paul Brown Stadium,"Cincinnati, Ohio",65515,2000,Cincinnati,Ohio
7,Cleveland Browns,First Energy Stadium,"Cleveland, Ohio",67895,1999,Cleveland,Ohio
8,Dallas Cowboys,At&T Stadium,"Arlington, Texas",80000,2009,Arlington,Texas
9,Denver Broncos,Empower Field at Mile High,"Denver, Colorado",76125,2001,Denver,Colorado


In [24]:
state_abbrevs = pd.read_pickle('./state_abbreviations.pkl')
state_abbrevs.to_dict()['Abbreviation']

{'Alabama': 'AL',
 'Alaska': 'AK',
 'Arizona': 'AZ',
 'Arkansas': 'AR',
 'California': 'CA',
 'Colorado': 'CO',
 'Connecticut': 'CT',
 'Delaware': 'DE',
 'Florida': 'FL',
 'Georgia': 'GA',
 'Hawaii': 'HI',
 'Idaho': 'ID',
 'Illinois': 'IL',
 'Indiana': 'IN',
 'Iowa': 'IA',
 'Kansas': 'KS',
 'Kentucky': 'KY',
 'Louisiana': 'LA',
 'Maine': 'ME',
 'Maryland': 'MD',
 'Massachusetts': 'MA',
 'Michigan': 'MI',
 'Minnesota': 'MN',
 'Mississippi': 'MS',
 'Missouri': 'MO',
 'Montana': 'MT',
 'Nebraska': 'NE',
 'Nevada': 'NV',
 'New Hampshire': 'NH',
 'New Jersey': 'NJ',
 'New Mexico': 'NM',
 'New York': 'NY',
 'North Carolina': 'NC',
 'North Dakota': 'ND',
 'Ohio': 'OH',
 'Oklahoma': 'OK',
 'Oregon': 'OR',
 'Pennsylvania': 'PA',
 'Rhode Island': 'RI',
 'South Carolina': 'SC',
 'South Dakota': 'SD',
 'Tennessee': 'TN',
 'Texas': 'TX',
 'Utah': 'UT',
 'Vermont': 'VT',
 'Virginia': 'VA',
 'Washington': 'WA',
 'West Virginia': 'WV',
 'Wisconsin': 'WI',
 'Wyoming': 'WY'}

In [38]:
def get_station_id(city, key):
    url = 'https://api.meteostat.net/v1/stations/search?q={}&key={}'.format(city, key)
    response = requests.get(url)
    return response.json()

In [39]:
# test historical weather API
city = 'Baltimore'
station = get_station_id(city, api_key)['data'][0]['id']
station

'72406'

In [42]:
for row in stadiums_df['City']:
    print(row, get_station_id(row, api_key))
    print('\n\n')

Glendale {'meta': {'source': 'National Oceanic and Atmospheric Administration, Deutscher Wetterdienst'}, 'data': [{'id': '72288', 'country': 'US', 'name': 'Burbank-Glendale-Pasadena'}, {'id': 'KGEU0', 'country': 'US', 'name': 'Glendale / Country Greens At Villa De Paz'}, {'id': 'KHVS0', 'country': 'US', 'name': 'Hartsville / Glendale Acres'}, {'id': 'KLUF0', 'country': 'US', 'name': 'Phoenix / Glendale / Webb'}, {'id': 'KORB0', 'country': 'US', 'name': 'Orr / Glendale'}]}



Atlanta {'meta': {'source': 'National Oceanic and Atmospheric Administration, Deutscher Wetterdienst'}, 'data': [{'id': '72219', 'country': 'US', 'name': 'Hartsfield-Jackson Atlanta'}, {'id': 'KCVC0', 'country': 'US', 'name': 'Atlanta'}, {'id': 'KFTY0', 'country': 'US', 'name': 'Atlanta / Carroll Heights'}, {'id': 'KPDK0', 'country': 'US', 'name': 'Atlanta / Northwoods'}]}



Baltimore {'meta': {'source': 'National Oceanic and Atmospheric Administration, Deutscher Wetterdienst'}, 'data': [{'id': '72406', 'country':

Oakland {'meta': {'source': 'National Oceanic and Atmospheric Administration, Deutscher Wetterdienst'}, 'data': [{'id': '72493', 'country': 'US', 'name': 'Metro Oakland International  Airport'}, {'id': 'KWVL0', 'country': 'US', 'name': 'Waterville / Oakland'}, {'id': 'KRJD0', 'country': 'US', 'name': 'Ridgely / Oakland Acres'}]}



Philadelphia {'meta': {'source': 'National Oceanic and Atmospheric Administration, Deutscher Wetterdienst'}, 'data': [{'id': '72408', 'country': 'US', 'name': 'Philadelphia International Airport'}, {'id': 'KPHD0', 'country': 'US', 'name': 'New Philadelphia / Schoenbrunn Estates'}, {'id': 'KPNE0', 'country': 'US', 'name': 'Philadelphia / Ashton Wooden Bridge'}]}



Pittsburgh {'meta': {'source': 'National Oceanic and Atmospheric Administration, Deutscher Wetterdienst'}, 'data': [{'id': '72520', 'country': 'US', 'name': 'Greater Pittsburgh International'}]}



Santa Clara {'meta': {'source': 'National Oceanic and Atmospheric Administration, Deutscher Wetterdie

In [ ]:
def get_historical_data(dirName, city, station_id, key, timezone, start, end):
    url = 'https://api.meteostat.net/v1/history/hourly?station={}&start={}&end={}&time_zone={}&time_format=Y-m-d%20H:i&key={}'
    response = requests.get(url.format(station_id,start,end,timezone,key))
    weather_data = response.json()
    weather_data = weather_data['data']
    
    with open('\{}\{data}.json'.format(dirName, city), 'w') as f:
        json.dump(weather_data, f)
    
    print('Successfully wrote to json for ', city, ' from', start, ' to ', end)